<h1 style="color:black" align="center">Детекция объектов</h1>

<!-- <h1 style="color:#008B8B">1. Non-maximum supression</h1> -->

## Non-maximum supression

* Модель выдаёт для класса $k$ список прямоугольников с уверенностями
* Проходим в порядке уменьшения уверенности
* Для каждого прямоугольника удаляем все последующие, с которыми Intersection over Union (IoU) > 0.5

Имеется $k$ классов и для каждого класса, модель выдаёт список прямоугольников с уверенностями. Возьмём первый прямоугольный (увверенность 0.9), из тех, которые модель выдала для первого класса. Среди всех следующих прямоугольников данного класса находим те, у которых (IoU) > 0.5. Переходим к следующему прямоугольники и повторяем операцию.

## Метрики качества

* Модель выдаёт для класса $k$ список прямоугольников с уверенностями
* Считаем прямоугольник корректным, если $\text{loU}(y,z) = \text{Jaccard}(y, z) > t$
* Строим PR-кривую, считаем под ней площадь, получаем Average Precision
* Усредняем по всем классам, получаем mAP (mean AP)
* Раньше $t = 0.5$, сейчас скорее $t = 0.75$

Имеется размеченная картинка, модель расставила на данной картинке некоторые прямоугольники. Каждый из прямоугольников модели считаются корректными, если $\text{loU}(y,z) = \text{Jaccard}(y, z) > t$. Благодоря данному порогу мы определяем, правлиьный ли прямоугольник.

Мы получили список прямоугольникоов, которые отсортированы по вероятностям принадлжености к некоторому классу. Дальше, проставляем метки правлиьности и считаем площадь под PR кривой. При построении PR кривой, мы сдвигаем порог, но используется порог на вероятности, которые выдает модель, а не loU.

<h1 style="color:#008B8B">1. Two-shot detection</h1>

Будем решать задачу в два этапа:
1. Находим «кандидатов» — прямоугольники, где скорее всего что-то есть
2. Классифицируем эти прямоугольники

## R-CNN

1. Имеется входная картинка.
2. Применяем, метод, которые формирует прямоугольники на картинке (~2000 штук). Берем каждый прямоугольник, выполняем преобразование его размера.
3. Данный прямоугольник передаём предобученной неройнной сети (AlexNet...), из него достаём признаки с предпоследнего слоя.
4. Используя данное признаковое описание обучаем классификатор (SVM...), который по данным признакм будет определять то, что находится в данном прямоугольнике.

<img src='img/lecture08/1.png'>

**Генерация кандидатов:**

* «Внешние» методы из классического компьютерного зрения
* Выбирается около 2000 прямоугольников

Используем методы классического компьютерного зрения. Берём все пиксели, для каждого пикслея берём его признаки, координаты, цвета... и кластеризуем все пиксели. Один кластер называется суперпикселем. Выбираем прямоугольники так, чтобы они покрывали все спуерпиксели.

<img src='img/lecture08/2.png'>

Классификация прямоугольников:
* SVM
* One-vs-all

### Проблемы:
* Не end-to-end
* Генерация кандидатов может быть очень сложной
* Свёрточная сеть практически не настраивается под данные
* Долго (много признаков, много классификаторов)

## Fast R-CNN

Кандидаты всё ещё генерируются внешним методом:

<img src='img/lecture08/3.png'>

* Для конкретного кандидата извлекаются признаки: вырезаются участки из последнего тензора, режутся на блоки, из каждого блока берётся максимум